## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-04-04-34 +0000,nyt,Here’s the latest on the tsunami.,https://www.nytimes.com/live/2025/07/29/world/...
1,2025-07-30-04-01-52 +0000,nyt,Reno Shooting Shatters Lives at a Bachelor Party,https://www.nytimes.com/2025/07/30/us/reno-cas...
2,2025-07-30-04-00-46 +0000,nyt,Ozzy Osbourne Funeral Procession to Be Held in...,https://www.nytimes.com/2025/07/29/arts/music/...
3,2025-07-30-04-00-41 +0000,wapo,Leading genocide scholars see a genocide happe...,https://www.washingtonpost.com/world/2025/07/3...
4,2025-07-30-04-00-07 +0000,nyt,How a Chinese Brand Reshaped Hong Kong’s Food ...,https://www.nytimes.com/2025/07/30/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,64
22,gaza,21
73,new,20
4,shooting,15
108,china,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
105,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,128
307,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,120
34,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,118
194,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,118
197,2025-07-29-17-38-03 +0000,nypost,UN-backed experts warn of ‘worst-case scenario...,https://nypost.com/2025/07/29/world-news/un-ba...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
105,128,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
32,72,2025-07-30-01-49-26 +0000,nypost,Reno casino victim ID’d as Calif. dad who advo...,https://nypost.com/2025/07/29/us-news/reno-cas...
237,66,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
34,54,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
65,53,2025-07-30-00-06-03 +0000,nypost,NYC skyscraper that turned into tower of terro...,https://nypost.com/2025/07/29/us-news/nyc-skys...
113,49,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...
160,48,2025-07-29-19-52-56 +0000,nyt,UK Will Recognize Palestinian Statehood In Sep...,https://www.nytimes.com/2025/07/29/world/europ...
30,44,2025-07-30-01-52-23 +0000,wapo,Trump softens stance on China amid trade talks...,https://www.washingtonpost.com/national-securi...
26,40,2025-07-30-02-12-08 +0000,nypost,"Trump tells Sen. Chuck Grassley to ‘step up,’ ...",https://nypost.com/2025/07/29/us-news/trump-te...
243,40,2025-07-29-15-07-33 +0000,latimes,Shane Tamura played high school football in L....,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
